# Up and going with tf( version 1)

## Imports

In [2]:
import tensorflow.compat.v1 as tf
import numpy
from sklearn.datasets import fetch_california_housing

2024-02-06 18:17:43.419147: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 18:17:44.018183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tf.disable_eager_execution() 

In [4]:
x = tf.Variable(3,name='x')
y = tf.Variable(8,name='y')

In [5]:
f = x*x*y + y + 5

In [6]:
sess = tf.Session()

In [7]:
sess.run(x.initializer)

2024-02-06 18:17:50.045584: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [8]:
sess.run(y.initializer)

In [9]:
result = sess.run(f)

In [10]:
result

85

In [11]:
sess.close()

In [12]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    res = sess.run(f)

In [13]:
res

85

In [14]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    res = f.eval()

In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    res = f.eval()

In [16]:
res

85

In [17]:
x1 = tf.Variable(1)

In [18]:
x1.graph

In [19]:
tf.get_default_graph()

In [20]:
x1.graph is tf.get_default_graph()

True

In [21]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(3)
    

In [22]:
x2.graph is graph

True

In [23]:
x2.graph is tf.get_default_graph()

False

In [24]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval()) 

10
15


In [25]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [26]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = numpy.c_[numpy.ones((m, 1)), housing.data]


In [27]:
m,n

(20640, 8)

In [28]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [29]:
scaled_housing_data_plus_bias = housing_data_plus_bias.copy()
means,stds = housing_data_plus_bias[:,1:].mean(axis=0),housing_data_plus_bias[:,1:].std(axis=0)
scaled_housing_data_plus_bias[:,1:] -= means
scaled_housing_data_plus_bias[:,1:] /= stds

In [30]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            sess.run(training_op)
            best_theta = theta.eval()

Epoch 0 MSE = 7.7471232
Epoch 100 MSE = 7.419629
Epoch 200 MSE = 7.1082563
Epoch 300 MSE = 6.812129
Epoch 400 MSE = 6.5304294
Epoch 500 MSE = 6.2623806
Epoch 600 MSE = 6.007256
Epoch 700 MSE = 5.7643743
Epoch 800 MSE = 5.5330896
Epoch 900 MSE = 5.312796


In [34]:
A = tf.placeholder(tf.float32,shape=(None,3))
B = A +5

with tf.Session() as sess:
    B1 = B.eval(feed_dict={A:[[1,2,3],[7,8,9]]})

In [35]:
B1

array([[ 6.,  7.,  8.],
       [12., 13., 14.]], dtype=float32)

# Mini Batch Gradient Descent

## Unscaled data( Exploding Gradient)

In [39]:
def fetch_mini_batch(ix,size):
    ix   = slice(size*ix,size*(ix+1))
    data = lambda : fetch_california_housing()
    y = data().target[ix].reshape(-1,1)
    X = data().data[ix,:]
    bias = numpy.ones((min(size,len(X)), 1))
    X = numpy.c_[bias, X]
    return y,X     

In [40]:
(r:=fetch_mini_batch(1,5),r[0].shape,r[1].shape)

((array([[2.697],
         [2.992],
         [2.414],
         [2.267],
         [2.611]]),
  array([[ 1.00000000e+00,  4.03680000e+00,  5.20000000e+01,
           4.76165803e+00,  1.10362694e+00,  4.13000000e+02,
           2.13989637e+00,  3.78500000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  3.65910000e+00,  5.20000000e+01,
           4.93190661e+00,  9.51361868e-01,  1.09400000e+03,
           2.12840467e+00,  3.78400000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  3.12000000e+00,  5.20000000e+01,
           4.79752705e+00,  1.06182380e+00,  1.15700000e+03,
           1.78825348e+00,  3.78400000e+01, -1.22250000e+02],
         [ 1.00000000e+00,  2.08040000e+00,  4.20000000e+01,
           4.29411765e+00,  1.11764706e+00,  1.20600000e+03,
           2.02689076e+00,  3.78400000e+01, -1.22260000e+02],
         [ 1.00000000e+00,  3.69120000e+00,  5.20000000e+01,
           4.97058824e+00,  9.90196078e-01,  1.55100000e+03,
           2.17226891e+00,  3.78400000e+01, -1.222

In [49]:
batch_size = 1000
n_batches  = int(numpy.ceil(m/batch_size))

In [50]:
y = tf.placeholder(tf.float32,shape=(None,1),name='y')
X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')

In [51]:
lr = .1

In [52]:
θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
yhat = tf.matmul(X,θ,name='predictions')

ε = yhat - y

mse = tf.reduce_mean(tf.square(ε),name='mse')

gradients = (2/m)*tf.matmul(tf.transpose(X),ε)

trainop = tf.assign(θ,θ-lr*gradients)

In [53]:
n_epochs = 200
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_ix in range(n_batches):
            y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
            sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={y:y_batch,X:X_batch}))
    best_theta = theta.eval()

Epoch 0 MSE = nan
Epoch 100 MSE = nan


## Scaled Data

In [54]:
def fetch_mini_batch(ix,size):
    ix   = slice(size*ix,size*(ix+1))
    data = lambda : fetch_california_housing()
    y = data().target[ix].reshape(-1,1)
    X = (data().data[ix,:]-means)/stds
    bias = numpy.ones((min(size,len(X)), 1))
    X = numpy.c_[bias, X]
    return y,X    

In [60]:
θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
yhat = tf.matmul(X,θ,name='predictions')

ε = yhat - y

mse = tf.reduce_mean(tf.square(ε),name='mse')

gradients = (2/m)*tf.matmul(tf.transpose(X),ε)

trainop = tf.assign(θ,θ-lr*gradients)

n_epochs = 1000
init = tf.global_variables_initializer()

saver = tf.train.Saver() #saving sessions

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        for batch_ix in range(n_batches):
            
            y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
            
            sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
            
        if epoch % 100 == 0:
                saver.save(sess,f"models/model.ckpt")
                print("Epoch", epoch, "MSE =", mse.eval(feed_dict={y:y_batch,X:X_batch}))
    bestθ = θ.eval()
    saver.save(sess,f"models/modelfinal.ckpt")


Epoch 0 MSE = 3.3881073
Epoch 100 MSE = 0.39172354
Epoch 200 MSE = 0.39354926
Epoch 300 MSE = 0.3944077
Epoch 400 MSE = 0.3944732
Epoch 500 MSE = 0.39443097
Epoch 600 MSE = 0.39439982
Epoch 700 MSE = 0.394385
Epoch 800 MSE = 0.39437777
Epoch 900 MSE = 0.3943751


In [58]:
best_theta

array([[-0.9616356 ],
       [ 0.40589   ],
       [-0.8566716 ],
       [-0.31369996],
       [-0.19853783],
       [ 0.49607372],
       [-0.51161814],
       [ 0.6872895 ],
       [ 0.7328303 ]], dtype=float32)

# Visializing with tensorboard

In [76]:
import datetime
now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
logd = f'tf_logs/run-{now}/'
logd

'tf_logs/run-2024-02-06-19-40-41/'

In [73]:
graph = tf.Graph()

with graph.as_default():
    
    y = tf.placeholder(tf.float32,shape=(None,1),name='y')
    X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')
    
    θ = tf.Variable(numpy.zeros((n+1,1),dtype='float32'),name='Theta')
    yhat = tf.matmul(X,θ,name='predictions')
    
    ε = yhat - y
    
    mse = tf.reduce_mean(tf.square(ε),name='mse')
    
    gradients = (1/m)*tf.matmul(tf.transpose(X),ε) #it's same as using 2, when minimizing it
    
    trainop = tf.assign(θ,θ-lr*gradients)
    
    n_epochs = 1000
    init = tf.global_variables_initializer()
        
    mse_summary = tf.summary.scalar('mse',mse)

    file_writer = tf.summary.FileWriter(logd)
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            
            for batch_ix in range(n_batches):
                
                y_batch,X_batch = fetch_mini_batch(batch_ix,batch_size)
                
                sess.run(trainop,feed_dict={y:y_batch,X:X_batch})
                
            if epoch % 100 == 0:
                summary = mse_summary.eval(feed_dict={y:y_batch,X:X_batch})
                file_writer.add_summary(summary,epoch)

        bestθ = θ.eval()

In [79]:
%ls tf_logs/

run-2024-02-06-19-10-02/


In [80]:
%ls tf_logs/run-2024-02-06-19-10-02/

events.out.tfevents.1707247181.0e6ab30c0699
events.out.tfevents.1707247840.0e6ab30c0699


In [81]:
file_writer

In [82]:
file_writer.close()

In [84]:
%ls -la tf_logs/run-2024-02-06-19-10-02/

total 16
drwxr-xr-x 2 analyst analyst 4096 Feb  6 19:30 ./
drwxr-xr-x 3 analyst analyst 4096 Feb  6 19:19 ../
-rw-r--r-- 1 analyst analyst  514 Feb  6 19:26 events.out.tfevents.1707247181.0e6ab30c0699
-rw-r--r-- 1 analyst analyst  514 Feb  6 19:51 events.out.tfevents.1707247840.0e6ab30c0699


In [ ]:
!tensorboard --logdir tf_logs/

2024-02-06 19:53:43.934472: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 19:53:44.199563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
